# __Datenbereinigung Teil 3: Bereinigung der Gesamtdaten__

In diesem Schritt werden die Gesamtdaten bereinigt.

In [1]:
from time import time
import pandas as pd 
from bs4 import BeautifulSoup
import string
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize.treebank import TreebankWordDetokenizer
import os

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Unterlagen/JobCloud_Daten')

In [3]:
# Einlesen der Daten über ein csv File
data = pd.read_csv("Inserate_Gesamtdatenanalyse_AF_E.csv", sep=";")
data.shape
#data = data.dropna()

print(data.head(5))
data.info()
#data.to_csv("Inserate_clean_Test.csv", sep=";")

   Unnamed: 0            ad_id active_start_on active_end_on  \
0           5  jobsse-10107990      2021-01-07    2021-01-15   
1           7   jobsse-8132125      2018-04-17    2018-04-18   
2          11   jobsse-6058207      2014-10-06    2014-11-07   
3          13   jobsse-5875311      2014-06-17    2014-06-20   
4          14   jobsse-6012460      2014-09-09    2014-09-16   

                                      title  \
0               Lead Solution Architect IoT   
1   Konstrukteur/In (auch Teilzeit möglich)   
2  Kaufmännische(n) Angestellte(n) 80%-100%   
3          Hauswirtschaftsmitarbeiter (m/w)   
4                     Kindergärtner/in 85 %   

                                                text  \
0  Lead Solution Architect IoT Als Lead Solution ...   
1  <p><strong>Ihre Hauptaufgaben</strong></p><ul>...   
2  <p><strong>Wir z&auml;hlen auf Sie:</strong></...   
3  Das Widder Hotel Moderne Architektur in 9 hist...   
4  Kindergärtner/in, Ref.-Nr. 41427D        Bezei...

In [4]:
# HTML-Elemente in einer neue Spalte 'Text_Anforderungen' abspeichern

start = time()

column = []
for i in data['text']:
    text=''
    soup = BeautifulSoup(i, 'lxml')
    text = soup.text 
    column.append(text)

data['Text_Anforderungen'] = column

print(data.Text_Anforderungen[0:5])
print('--------------------------')
print(data.shape)
print('--------------------------')
print(f'Benötigte Zeit: {time() - start} Sekunden')

C:\Users\Sven Konermann\anaconda3\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: "/" looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\Sven Konermann\anaconda3\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: "..." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


0    Lead Solution Architect IoT Als Lead Solution ...
1    Ihre HauptaufgabenTeile in verschiedenen Softw...
2    Wir zählen auf Sie:Sie führen einen Teilbereic...
3    Das Widder Hotel Moderne Architektur in 9 hist...
4    Kindergärtner/in, Ref.-Nr. 41427D        Bezei...
Name: Text_Anforderungen, dtype: object
--------------------------
(134074, 15)
--------------------------
Benötigte Zeit: 130.97111988067627 Sekunden


In [5]:
# Lemmatisierung der Gesamtdaten (kann je nach Datenmenge länger dauern).
from HanTa import HanoverTagger as ht
start = time()
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

for mail in data['Text_Anforderungen']:
    lemma = [lemma for (word,lemma,pos) in tagger.tag_sent(mail.split())]
    index = data[data['Text_Anforderungen']==mail].index
    data.loc[index, 'Text_Anforderungen'] = (' '.join(lemma))
    
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 27883.0457572937 Sekunden


In [6]:
# Text in Kleinbuchstaben umwandeln
data['Text_Anforderungen'] = (data['Text_Anforderungen'].str.lower())

In [7]:
# Satzzeichen löschen
def remove_punctuation(txt):
    txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct

data['Text_Anforderungen'] = data['Text_Anforderungen'].apply(lambda x: remove_punctuation(x))

In [8]:
# \t-Tag entfernen aus den Texten
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("\t", "", regex=True)

In [9]:
# Manuell ähnliche Anforderungen zu einem Hauptbegriff ersetzen
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("teamplayer", "teamfähig", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("verhandlungssicher", "verhandlungsgeschick", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kundenorientierte", "kundenorientierung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("projekt management", "projektleitungserfahrung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("projektmanagement", "projektleitungserfahrung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("zusammenarbeit", "teamfähig", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kommunikation", "kommunikationsfähigkeit", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kommunikationsfähigkeiten", "kommunikationsfähigkeit", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("dwh", "data warehouse", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("dienstleistungsorientierung", "dienstleistungsorientiert", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("eigenverantwortung", "eigenverantwortlich", regex=True)

In [10]:
# Word-Tokenization 
start = time()
data['text_tok'] = data.apply(lambda row: nltk.word_tokenize(row['Text_Anforderungen']), axis=1)
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 82.30526733398438 Sekunden


In [11]:
# Stopwörter löschen
stop_words = set(stopwords.words('german'))
data['text_tok'] = data['text_tok'].apply(lambda x: [item for item in x if item not in stop_words])

In [12]:
# Tokenizing rückgängig machen, da die Tokenizierung nicht in eine csv-Datei überführt werden kann
column = []
for i in data['text_tok']:
    test = TreebankWordDetokenizer().detokenize(i)
    column.append(test)

data['Text_Anforderungen'] = column

In [13]:
# Löschen der tokenisierten Spalte
data = data.drop(['text_tok'], axis = 1)

In [14]:
# Erneutes Löschen der Duplikate, da sich die Texte verändert haben
data = data.drop_duplicates(subset='Text_Anforderungen', keep="last")

In [15]:
data.to_csv("Inserate_clean_Gesamtdatenanalyse.csv", sep=";")

nächster Schritt: Mithilfe der erstellten Wörterbücher, die Gesamtdatenanalyse durchführen (Gesamtdatenanalyse.ipynb)